<a href="https://colab.research.google.com/github/ysuter/FHNW-BAI-DeepLearning/blob/main/Interpretabilitymaps_captum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔍 Saliency Maps & Interpretierbarkeit mit Captum

**Lernziele:**
- Verstehen, was Saliency Maps zeigen und was nicht
- Verschiedene Attributionsmethoden kennenlernen (Saliency, Integrated Gradients, GradCAM)
- Kritisch reflektieren: Sind diese Erklärungen "echt"?

---

## 1. Setup & Installation

In [ ]:
# Captum installieren (Facebooks Interpretability Library für PyTorch)
!pip install captum -q
!pip install numpy==2.2

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt

# Captum Imports
from captum.attr import Saliency, IntegratedGradients, GuidedGradCam, LayerGradCam
from captum.attr import visualization as viz

# Für reproduzierbare Ergebnisse
torch.manual_seed(42)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Verwendetes Device: {device}")

## 2. Modell laden

Wir verwenden **ResNet-18** – ein relativ kleines CNN mit 18 Schichten. Es wurde auf ImageNet (1000 Klassen, 1.2 Mio. Bilder) trainiert.

In [ ]:
# Vortrainiertes ResNet-18 laden
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model = model.to(device)
model.eval()  # Wichtig: Evaluation-Modus für Inferenz

print(f"Modell geladen: {sum(p.numel() for p in model.parameters()):,} Parameter")

In [ ]:
# ImageNet Klassennamen laden
LABELS_URL = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
imagenet_classes = requests.get(LABELS_URL).text.strip().split("\n")
print(f"{len(imagenet_classes)} Klassen geladen. Beispiele: {imagenet_classes[:5]}")

## 3. Hilfsfunktionen

In [ ]:
# Preprocessing für ImageNet-Modelle
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

def load_image(url_or_path):
    """Lädt ein Bild von URL oder lokalem Pfad."""
    if url_or_path.startswith('http'):
        response = requests.get(url_or_path)
        img = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        img = Image.open(url_or_path).convert('RGB')
    return img

def predict(model, input_tensor):
    """Gibt Top-5 Vorhersagen zurück."""
    with torch.no_grad():
        output = model(input_tensor)
        probs = F.softmax(output, dim=1)
        top5_probs, top5_indices = probs.topk(5)

    results = []
    for prob, idx in zip(top5_probs[0], top5_indices[0]):
        results.append((imagenet_classes[idx], prob.item()))
    return results, top5_indices[0][0].item()

def tensor_to_image(tensor):
    """Konvertiert normalisierten Tensor zurück zu anzeigbarem Bild."""
    # Denormalisieren
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    img = tensor.cpu() * std + mean
    img = img.squeeze().permute(1, 2, 0).detach().numpy()
    return np.clip(img, 0, 1)

## 4. Testbild laden

Wir starten mit einem klassischen Beispiel. Später könnt ihr eigene Bilder testen!

In [ ]:
# Beispielbilder zum Testen (einfach URL ändern oder eigenes Bild hochladen)
EXAMPLE_IMAGES_old = {
    "hund": "https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/YellowLabradorLooking_new.jpg/1200px-YellowLabradorLooking_new.jpg",
    "katze": "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg",
    "auto": "https://upload.wikimedia.org/wikipedia/commons/thumb/1/1b/2019_Toyota_Corolla_Icon_Tech_VVT-i_Hybrid_1.8.jpg/1200px-2019_Toyota_Corolla_Icon_Tech_VVT-i_Hybrid_1.8.jpg",
    "vogel": "https://upload.wikimedia.org/wikipedia/commons/thumb/4/45/Eopsaltria_australis_-_Mogo_Campground.jpg/1200px-Eopsaltria_australis_-_Mogo_Campground.jpg",
    "schmetterling": "https://upload.wikimedia.org/wikipedia/commons/thumb/6/6d/Papilio_machaon_Mitterbach_01.jpg/1200px-Papilio_machaon_Mitterbach_01.jpg"
}

EXAMPLE_IMAGES = {
    "hund": "https://images.unsplash.com/photo-1587300003388-59208cc962cb?w=800",
    "katze": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=800",
    "auto": "https://images.unsplash.com/photo-1494976388531-d1058494ceb8?w=800",
    "vogel": "https://images.unsplash.com/photo-1444464666168-49d633b86797?w=800",
}

# Bild auswählen
selected_image = "katze"  # <-- HIER GEÄNDERT, um ein anderes Bild zu versuchen

# Bild laden und vorbereiten
original_image = load_image(EXAMPLE_IMAGES[selected_image])
input_tensor = preprocess(original_image).unsqueeze(0).to(device)
input_tensor.requires_grad = True  # Wichtig für Gradient-basierte Methoden!

# Vorhersage
predictions, top_class = predict(model, input_tensor)

# Anzeigen
plt.figure(figsize=(8, 6))
plt.imshow(original_image)
plt.title(f"Top-Vorhersage: {predictions[0][0]} ({predictions[0][1]*100:.1f}%)")
plt.axis('off')
plt.show()

print("\nTop-5 Vorhersagen:")
for label, prob in predictions:
    print(f"  {label}: {prob*100:.2f}%")

---

## 5. Attributionsmethoden

Jetzt wird es spannend: Wir fragen das Modell "Warum hast du so entschieden?"

### 5.1 Vanilla Saliency (Gradient)

**Idee:** Berechne den Gradienten des Outputs bezüglich des Inputs. Pixel mit hohem Gradienten sind "wichtig" für die Entscheidung.



In [ ]:
# Saliency berechnen
saliency = Saliency(model)
attribution_saliency = saliency.attribute(input_tensor, target=top_class)

# Visualisierung
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Originalbild
axes[0].imshow(tensor_to_image(input_tensor))
axes[0].set_title("Original")
axes[0].axis('off')

# Saliency Map (Graustufen)
saliency_np = attribution_saliency.squeeze().cpu().detach().numpy()
saliency_gray = np.abs(saliency_np).max(axis=0)  # Max über Farbkanäle
axes[1].imshow(saliency_gray, cmap='hot')
axes[1].set_title("Saliency Map")
axes[1].axis('off')

# Overlay
axes[2].imshow(tensor_to_image(input_tensor))
axes[2].imshow(saliency_gray, cmap='hot', alpha=0.5)
axes[2].set_title("Overlay")
axes[2].axis('off')

plt.suptitle(f"Vanilla Saliency für Klasse: {imagenet_classes[top_class]}", fontsize=14)
plt.tight_layout()
plt.show()

### 💬 Diskussionsfrage

> Schaut euch die Saliency Map an: Fokussiert das Modell auf die "richtigen" Stellen? Oder gibt es überraschende Bereiche?

---

### 5.2 Integrated Gradients

**Problem mit Vanilla Saliency:** Gradienten können bei gesättigten Funktionen verschwinden (Gradient Saturation).

**Lösung:** Integriere Gradienten entlang eines Pfades von einer Baseline (z.B. schwarzes Bild) zum Input.

**Formel:** $\text{IG}(x) = (x - x') \cdot \int_0^1 \frac{\partial f(x' + \alpha(x-x'))}{\partial x} d\alpha$

Diese Methode erfüllt wichtige **Axiome**:
- Sensitivity: Unterschiedliche Inputs → unterschiedliche Attributionen
- Implementation Invariance: Gleiche Funktion → gleiche Attribution

In [ ]:
# Integrated Gradients berechnen
ig = IntegratedGradients(model)

# Baseline: schwarzes Bild (Standard), alternativ: Rauschen, Blur, etc.
baseline = torch.zeros_like(input_tensor).to(device)

attribution_ig = ig.attribute(
    input_tensor,
    baselines=baseline,
    target=top_class,
    n_steps=50  # Anzahl der Integrationsschritte
)

# Visualisierung
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(tensor_to_image(input_tensor))
axes[0].set_title("Original")
axes[0].axis('off')

ig_np = attribution_ig.squeeze().cpu().detach().numpy()
ig_gray = np.abs(ig_np).max(axis=0)
axes[1].imshow(ig_gray, cmap='hot')
axes[1].set_title("Integrated Gradients")
axes[1].axis('off')

axes[2].imshow(tensor_to_image(input_tensor))
axes[2].imshow(ig_gray, cmap='hot', alpha=0.5)
axes[2].set_title("Overlay")
axes[2].axis('off')

plt.suptitle(f"Integrated Gradients für Klasse: {imagenet_classes[top_class]}", fontsize=14)
plt.tight_layout()
plt.show()

### 💡 Experiment: Andere Baseline ausprobieren

Die Wahl der Baseline beeinflusst die Attribution! Testet verschiedene Baselines:

In [ ]:
# Verschiedene Baselines vergleichen
baselines = {
    "Schwarz": torch.zeros_like(input_tensor),
    "Weiss": torch.ones_like(input_tensor),
    "Rauschen": torch.randn_like(input_tensor) * 0.1,
    "Blur": transforms.GaussianBlur(kernel_size=31, sigma=10)(input_tensor.cpu()).to(device)
}

fig, axes = plt.subplots(1, len(baselines), figsize=(16, 4))

for ax, (name, baseline) in zip(axes, baselines.items()):
    attr = ig.attribute(input_tensor, baselines=baseline.to(device), target=top_class, n_steps=30)
    attr_np = np.abs(attr.squeeze().cpu().detach().numpy()).max(axis=0)
    ax.imshow(tensor_to_image(input_tensor))
    ax.imshow(attr_np, cmap='hot', alpha=0.5)
    ax.set_title(f"Baseline: {name}")
    ax.axis('off')

plt.suptitle("Einfluss der Baseline auf Integrated Gradients", fontsize=14)
plt.tight_layout()
plt.show()

### 💬 Diskussionsfrage

> Welche Baseline ist die "richtige"? Gibt es überhaupt eine richtige Antwort?

---

### 5.3 GradCAM (Gradient-weighted Class Activation Mapping)

**Idee:** Statt Pixel-level Attributionen schauen wir uns an, welche **Feature Maps** in einer bestimmten Schicht wichtig sind.

**Vorteil:** Gröbere, aber oft intuitivere Erklärungen. Zeigt "Regionen" statt einzelner Pixel.

In [ ]:
# GradCAM auf der letzten Convolutional Layer
gradcam = LayerGradCam(model, model.layer4[-1].conv2)  # Letzte Conv-Schicht von ResNet

attribution_gradcam = gradcam.attribute(input_tensor, target=top_class)

# GradCAM muss hochskaliert werden (ist nur 7x7 bei ResNet)
gradcam_upsampled = F.interpolate(
    attribution_gradcam,
    size=(224, 224),
    mode='bilinear',
    align_corners=False
)

# Visualisierung
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(tensor_to_image(input_tensor))
axes[0].set_title("Original")
axes[0].axis('off')

gradcam_np = gradcam_upsampled.squeeze().cpu().detach().numpy()
gradcam_np = np.maximum(gradcam_np, 0)  # ReLU
axes[1].imshow(gradcam_np, cmap='jet')
axes[1].set_title("GradCAM Heatmap")
axes[1].axis('off')

axes[2].imshow(tensor_to_image(input_tensor))
axes[2].imshow(gradcam_np, cmap='jet', alpha=0.5)
axes[2].set_title("Overlay")
axes[2].axis('off')

plt.suptitle(f"GradCAM für Klasse: {imagenet_classes[top_class]}", fontsize=14)
plt.tight_layout()
plt.show()

---

## 6. Methodenvergleich

Alle drei Methoden auf einen Blick:

In [ ]:
def compare_methods(input_tensor, target_class):
    """Vergleicht alle Attributionsmethoden nebeneinander."""

    # Berechnungen
    saliency_attr = saliency.attribute(input_tensor, target=target_class)
    ig_attr = ig.attribute(input_tensor, target=target_class, n_steps=50)
    gradcam_attr = gradcam.attribute(input_tensor, target=target_class)
    gradcam_up = F.interpolate(gradcam_attr, size=(224, 224), mode='bilinear', align_corners=False)

    # Normalisieren
    sal_np = np.abs(saliency_attr.squeeze().cpu().detach().numpy()).max(axis=0)
    ig_np = np.abs(ig_attr.squeeze().cpu().detach().numpy()).max(axis=0)
    gc_np = np.maximum(gradcam_up.squeeze().cpu().detach().numpy(), 0)

    # Plot
    fig, axes = plt.subplots(1, 4, figsize=(18, 5))

    axes[0].imshow(tensor_to_image(input_tensor))
    axes[0].set_title(f"Original\n{imagenet_classes[target_class]}")
    axes[0].axis('off')

    axes[1].imshow(tensor_to_image(input_tensor))
    axes[1].imshow(sal_np, cmap='hot', alpha=0.6)
    axes[1].set_title("Saliency\n(Gradient)")
    axes[1].axis('off')

    axes[2].imshow(tensor_to_image(input_tensor))
    axes[2].imshow(ig_np, cmap='hot', alpha=0.6)
    axes[2].set_title("Integrated Gradients\n(Pfad-Integration)")
    axes[2].axis('off')

    axes[3].imshow(tensor_to_image(input_tensor))
    axes[3].imshow(gc_np, cmap='jet', alpha=0.5)
    axes[3].set_title("GradCAM\n(Feature-Map-basiert)")
    axes[3].axis('off')

    plt.tight_layout()
    plt.show()

compare_methods(input_tensor, top_class)

---

## 7. 🧪 Experimente zum Selbst-Ausprobieren

### Experiment A: Andere Klasse als Ziel

Was, wenn wir fragen: "Wo im Bild sieht das Modell Hinweise auf eine ANDERE Klasse?"

In [ ]:
# Wähle eine andere Klasse als Ziel
# Finde Index für eine bestimmte Klasse:
def find_class(name):
    matches = [(i, c) for i, c in enumerate(imagenet_classes) if name.lower() in c.lower()]
    return matches[:10]

# Suche nach Klassen
print("Beispiel-Klassen mit 'cat':")
print(find_class("cat"))

print("\nBeispiel-Klassen mit 'grass':")
print(find_class("grass"))

In [ ]:
# Vergleiche: Attribution für tatsächliche Klasse vs. andere Klasse
alternative_class = 281  # 281 = tabby cat (bei Hundebild interessant)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# GradCAM für Top-Prediction
attr_top = gradcam.attribute(input_tensor, target=top_class)
attr_top_up = F.interpolate(attr_top, size=(224, 224), mode='bilinear', align_corners=False)
axes[0].imshow(tensor_to_image(input_tensor))
axes[0].imshow(np.maximum(attr_top_up.squeeze().cpu().detach().numpy(), 0), cmap='jet', alpha=0.5)
axes[0].set_title(f"GradCAM für: {imagenet_classes[top_class]}")
axes[0].axis('off')

# GradCAM für alternative Klasse
attr_alt = gradcam.attribute(input_tensor, target=alternative_class)
attr_alt_up = F.interpolate(attr_alt, size=(224, 224), mode='bilinear', align_corners=False)
axes[1].imshow(tensor_to_image(input_tensor))
axes[1].imshow(np.maximum(attr_alt_up.squeeze().cpu().detach().numpy(), 0), cmap='jet', alpha=0.5)
axes[1].set_title(f"GradCAM für: {imagenet_classes[alternative_class]}")
axes[1].axis('off')

plt.tight_layout()
plt.show()

### Experiment B: Eigenes Bild hochladen

In [ ]:
# In Google Colab: Bild hochladen
from google.colab import files

print("Lade ein Bild hoch...")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    custom_image = load_image(filename)
    custom_tensor = preprocess(custom_image).unsqueeze(0).to(device)
    custom_tensor.requires_grad = True

    predictions, custom_top_class = predict(model, custom_tensor)
    print(f"\nVorhersage: {predictions[0][0]} ({predictions[0][1]*100:.1f}%)")

    compare_methods(custom_tensor, custom_top_class)

### Experiment C: Andere Schicht für GradCAM

In [ ]:
# GradCAM auf verschiedenen Schichten von ResNet
layers = {
    "Layer 1 (früh)": model.layer1[-1].conv2,
    "Layer 2": model.layer2[-1].conv2,
    "Layer 3": model.layer3[-1].conv2,
    "Layer 4 (spät)": model.layer4[-1].conv2,
}

fig, axes = plt.subplots(1, len(layers) + 1, figsize=(20, 4))

axes[0].imshow(tensor_to_image(input_tensor))
axes[0].set_title("Original")
axes[0].axis('off')

for ax, (name, layer) in zip(axes[1:], layers.items()):
    gc = LayerGradCam(model, layer)
    attr = gc.attribute(input_tensor, target=top_class)
    attr_up = F.interpolate(attr, size=(224, 224), mode='bilinear', align_corners=False)

    ax.imshow(tensor_to_image(input_tensor))
    ax.imshow(np.maximum(attr_up.squeeze().cpu().detach().numpy(), 0), cmap='jet', alpha=0.5)
    ax.set_title(name)
    ax.axis('off')

plt.suptitle("GradCAM auf verschiedenen Netzwerk-Tiefen", fontsize=14)
plt.tight_layout()
plt.show()

### 💬 Diskussionsfrage

> Frühe Schichten zeigen feinere Details, späte Schichten größere Regionen. Welche Erklärung ist "besser"?

---

## 8. Kritische Reflexion

### ⚠️ Grenzen von Saliency Maps

1. **Keine Kausalität:** Saliency Maps zeigen Korrelationen, nicht Ursachen
2. **Instabilität:** Kleine Änderungen im Bild → große Änderungen in der Map
3. **Faithfulness:** Erklärt die Map wirklich das Modell oder nur unsere Erwartungen?
4. **Baseline-Abhängigkeit:** Bei IG beeinflusst die Baseline das Ergebnis massiv

In [ ]:
# Demo: Instabilität von Saliency Maps
# Kleines Rauschen hinzufügen
noise = torch.randn_like(input_tensor) * 0.01
noisy_tensor = (input_tensor + noise).requires_grad_(True)

# Saliency berechnen
attr_original = saliency.attribute(input_tensor, target=top_class)
attr_noisy = saliency.attribute(noisy_tensor, target=top_class)

# Vergleich
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

sal_orig = np.abs(attr_original.squeeze().cpu().detach().numpy()).max(axis=0)
sal_noisy = np.abs(attr_noisy.squeeze().cpu().detach().numpy()).max(axis=0)

axes[0].imshow(sal_orig, cmap='hot')
axes[0].set_title("Saliency: Original")
axes[0].axis('off')

axes[1].imshow(sal_noisy, cmap='hot')
axes[1].set_title("Saliency: +1% Rauschen")
axes[1].axis('off')

axes[2].imshow(np.abs(sal_orig - sal_noisy), cmap='hot')
axes[2].set_title("Differenz")
axes[2].axis('off')

plt.suptitle("⚠️ Instabilität: Minimales Rauschen → Andere Erklärung", fontsize=14)
plt.tight_layout()
plt.show()

---

## 9. Zusammenfassung

| Methode | Vorteile | Nachteile |
|---------|----------|------------|
| **Saliency** | Schnell, einfach | "Verrauscht", instabil |
| **Integrated Gradients** | Theoretisch fundiert, Axiome | Baseline-abhängig, langsamer |
| **GradCAM** | Intuitive Regionen, stabil | Grob, nur für CNNs |

### 🎯 Key Takeaways

1. **Keine Methode ist perfekt** – alle haben Trade-offs
2. **Post-hoc ≠ echtes Verstehen** – wir approximieren nur
3. **Mehrere Methoden kombinieren** für robustere Insights
4. **Domain-Experten einbeziehen** – nur sie können Plausibilität bewerten

---

## 10. Weiterführende Ressourcen

- [Captum Dokumentation](https://captum.ai/)
- [Paper: "Axiomatic Attribution for Deep Networks"](https://arxiv.org/abs/1703.01365) (Integrated Gradients)
- [Paper: "Grad-CAM"](https://arxiv.org/abs/1610.02391)
- [Paper: "Stop Explaining Black Box ML Models..."](https://arxiv.org/abs/1811.10154) (Kritik an Post-hoc Erklärungen)